jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import random
import copy
import heapq
import math

In [ ]:
df = pd.read_csv('C:/Projects IT/Python/Projet Metaheuristic/project.csv', sep =';')
nameTable = pd.read_csv('C:/Projects IT/Python/Projet Metaheuristic/indexStation.csv', sep =';')

df.head(3)

In [ ]:
nameTable.head(3)

In [ ]:
MG = nx.MultiGraph()

In [ ]:
for i in range(0, len(df.Station)):
    MG.add_edge(df.Station[i], df.Connection[i], weight=df.Time[i])

In [ ]:
MG

In [ ]:
print(MG.adj)

In [ ]:

plt.figure(figsize=(12,12))
nx.draw(MG, label=False)

plt.show()

In [ ]:
shortestMulti = nx.johnson(MG)

In [ ]:
shortestMulti[1]

In [ ]:
shortestMulti[0][1][0]

In [ ]:
class Chromosome:
    def __init__(self, nodes):
        self.nodes = nodes
        
    
    def shuffleGenes(self):
        random.shuffle(self.nodes)   
    
    #Order Changing Mutation, 1 gene is selected thanks to Pm probability then swapped with the other node(cant duplicate)
    def mutationGene(self, Pm):
        self.Pm = Pm
        for i in range(0, len(self.nodes)):
            self.randMutate = random.random()
            if self.randMutate < self.Pm:
                self.mutateTo = random.randint(0,len(self.nodes)-1)
                self.tmpGene = copy.copy(self.nodes[i])
                self.nodes = np.where(self.nodes == self.mutateTo, self.tmpGene, self.nodes)
                self.nodes[i] = self.mutateTo

    def pathWay(self, Multigraph): #Multigraph come from the Johnson Networkx calculation for shortest path between 2 nodes
        self.Multigraph = Multigraph
        self.totalPath = []
        for i in range(0, len(self.nodes)-1):        
            if i+1 is not None:
                self.totalPath = merge(self.totalPath, self.Multigraph[self.nodes[i]][self.nodes[i+1]])   
        return self.totalPath
        
    def __repr__(self):
        self.strNodes = "Chromosome :"
        for i in self.nodes:
            self.node = str(i)
            self.strNodes = self.strNodes + "|" + self.node
        self.strNodes = self.strNodes + "|"
        return self.strNodes

In [ ]:
#Crossover PErmutation Encoding (1)ABC DEF + (2)DAB CFE --> ABC(1) DFE (in the order of the 2d chromosome genes)
def crossoverPermutation(chromo1, chromo2):
    lenChromo = int(len(chromo1.nodes) / 2)
    childrenChr = chromo1.nodes[0:lenChromo]
    for i in chromo2.nodes:
        if i not in childrenChr:
            childrenChr = np.append(childrenChr, [i])
    return childrenChr
    

In [ ]:
#Crossover PErmutation Encoding (1)ABC DEF + (2)DAB CFE --> ABC(1) DFE (in the order of the 2d chromosome genes)
def crossoverSinglePoint(chromo1, chromo2):
    lenChromo = int(len(chromo1.nodes) / 2)
    childrenChr = np.append(chromo1.nodes[0:lenChromo], chromo2.nodes[lenChromo:len(chromo1.nodes)])
    return childrenChr
    

In [ ]:
a = np.array([1,2,3,4,5,6,7,8,9,0])
b = np.array([11,22,33,44,55,66,77,88,99,100])
gen1 = Chromosome(a)
gen2 = Chromosome(b)
child = Chromosome(crossoverSinglePoint(gen2,gen1))
child

In [ ]:
def populationGenerationStart(popsize, nodeList, start):
    populationOpti = np.array([])
    for i in range(0,popsize,1):
        nodeList = np.unique(nodeList, axis=0)
        random.shuffle(nodeList)
        nodeList = np.where(nodeList == start, nodeList[0], nodeList)
        nodeList[0] = start
        populationOpti = np.append(populationOpti, Chromosome(nodeList))
    return populationOpti

In [ ]:
def populationGenerationClassic(popsize, nodeList):
    populationOpti = []
    for i in range(0,popsize,1):
        nodeList = np.unique(nodeList, axis=0)
        random.shuffle(nodeList)
        populationOpti = populationOpti + [Chromosome()]
    return populationOpti

In [ ]:
#function can merge 2 list where last element of one(a) is the first element of the second(b)
#for example [1,2,3,4] + [4,5,6] = [1,2,3,4,5,6]
def merge(a, b):
    c = a[:-1] + b[:]
    return c
    

In [ ]:
def NewFit(uniqueNodeList, chromosome, MultiG):
    if len(chromosome.pathWay(MultiG)) < 1000:
        fitScore = (len(np.unique(chromosome.pathWay(MultiG))) / (len(uniqueNodeList.unique())) )
    else:
        fitScore = 0
    return fitScore
    
    

In [ ]:
def NewTimeFit(uniqueNodeList, chromosome, MultiG):
    if len(chromosome.pathWay(MultiG)) < 1500:
        fitScore = (len(np.unique(chromosome.pathWay(MultiG))) / (len(uniqueNodeList.unique())) 
    else:
        fitScore = 0
    return fitScore

In [ ]:
def NewNameFit(uniqueNodeName, chromosome, MultiG):
    if len(chromosome.pathWay(MultiG)) < 1000:
        NameList = np.array([])
        for i in chromosome.pathWay(MultiG):
            NameList = np.append(NameList, uniqueNodeName[i])
        fitScore = len(np.unique(NameList)) / 296
    else:
        fitScore = 0
    return fitScore
    
    

In [ ]:
def fitnessScore(uniqueNodeList, chromosome, MultiG):
    fitScore = len(uniqueNodeList.unique()) / len(chromosome.pathWay(MultiG))
    return fitScore

In [ ]:
def nameFit(uniqueNodeName, chromosome, MultiG):
    NameList = np.array([])
    for i in chromosome.pathWay(MultiG):
        NameList = np.append(NameList, uniqueNodeName[i])
    NameFreq = np.unique(NameList, return_counts=True)
    NameFreq = np.sort(NameFreq[1])
    score =  296 / len(NameList)
    return score

In [ ]:
a = np.array(nameTable.nameStation)
len(np.unique(a))

In [ ]:
def nodesFit(chromosome, MultiG):
    score = 1 / len(chromosome.pathWay(MultiG))
    return score
    

In [ ]:
def betweenNodesFit(chromFct, MG):
    scoreFinal = 0
    for i in range(0, len(chromFct.nodes)-1):
        if i+1 is not None:
            lenEdges = len(MG[chromFct.nodes[i]][chromFct.nodes[i+1]])
            scoreEdge = 1 / ((lenEdges)**2)
            scoreFinal = scoreEdge + scoreFinal
    return scoreFinal
            

In [ ]:
'''
def mainIterOptimization(popLocal, popsize, nbIter):
    for z in range(0,nbIter):
        fitList = np.array([])
        for i in range(0,len(popLocal)):
            fitList = np.append(fitList, fitnessScore(df.Station, popLocal[i], shortestMulti))
        IndexfitBest = heapq.nlargest(int(popsize / 2), range(len(fitList)), fitList.take)
        popLocal_best = np.take(popLocal, IndexfitBest)
        child = np.array([Chromosome(crossoverPermutation(popLocal_best[0], popLocal_best[int(popsize/2) - 1]))])
        for u in range(1, len(popLocal_best)):
            crossChr = Chromosome(crossoverPermutation(popLocal_best[u-1], popLocal_best[u]))
            if crossChr not in child:
                child = np.append(child, crossChr)
            else:
                crossChr.mutationGene(0.2)
                child = np.append(child, crossChr)
        popLocal = np.append(child, popLocal_best)
        for mutIter in range(0, len(popLocal)-1):
            rand = random.random()
            if rand < 0.2:
                popLocal[mutIter].mutationGene(0.1)
    return popLocal[1]
'''

In [ ]:
fitList = np.array([])
for i in range(0,len(newNewnewPop)):
            fitList = np.append(fitList, fitnessScore(df.Station, newNewnewPop[i], shortestMulti))
fitList

In [ ]:
#station = 325 start,268
#BEST 3611 nodes
#MAIN LOOP
population = ([])
startGene = 268

FirstFitList = np.array([])

for i in range(0, 100):
    tmpNode = np.random.randint(375, size = (50))
    tmpNode[0] = startGene
    tmpC = Chromosome(tmpNode)
    population = np.append(population, tmpC)
    FirstFitList = np.append(FirstFitList, NewNameFit(nameTable.nameStation, population[i], shortestMulti))
print(FirstFitList)
X_opti = np.array([0])
Y_opti = np.array([np.max(FirstFitList)])
for z in range(0,50000):
    fitList = np.array([])
    for i in range(0,len(population)):
        fitList = np.append(fitList, NewNameFit(nameTable.nameStation, population[i], shortestMulti))
    IndexfitBest = heapq.nlargest(10, range(len(fitList)), fitList.take)
    population_best = np.take(population, IndexfitBest)
    child = np.array([Chromosome(crossoverPermutation(population_best[0], population_best[9]))])
    for u in range(1, len(population_best)):
        crossChr = Chromosome(crossoverPermutation(population_best[u-1], population_best[u]))
        if crossChr not in child:
            child = np.append(child, crossChr)
        elif NewFit(df.Station, population_best[u], shortestMulti) < 0.1:
            crossChr.mutationGene(1)
            child = np.append(child, crossChr)
        else:
            crossChr.mutationGene(0.05)
            child = np.append(child, crossChr)
            #child = np.append(child, Chromosome(df.Station).shuffleGenes())
    population = np.append(child, population_best)
    for mutIter in range(0, len(population)-1):
        rand = random.random()
        if rand < 0.2:
            population[mutIter].mutationGene(0.05)
   # bestChromoIter = population[1]
   # newPopulation = np.append(newPopulation,bestChromoIter)
    print(np.max(fitList))
    X_opti = np.append(X_opti, z)
    Y_opti = np.append(Y_opti, np.max(fitList))

print(newPopulation)


In [ ]:
for i in range(0,5):
    print(i)

In [ ]:
#NEW ALGO split avec duplicate CROSSOVER SINGLE POINT
#MAIN LOOP
population = np.array([])
#startGene = 268
X_opti = np.array([])
Y_opti = np.array([])

for i in range(0, 50):
    tmpNode = np.random.randint(375, size = (50))
    #tmpNode[0] = startGene
    #print(tmpNode)
    tmpC = Chromosome(tmpNode)
    population = np.append(population, tmpC)
    
for z in range(0,50000):
    fitList = np.array([])
    for i in range(0,len(population)):
        fitList = np.append(fitList, NewNameFit(nameTable.nameStation, population[i], shortestMulti))
    IndexfitBest = heapq.nlargest(20, range(len(fitList)), fitList.take)
    population_best = np.take(population, IndexfitBest)
    child = np.array([population_best[0], population_best[1]]) #2 best chromosomes kept
    for u in range(1, len(population_best)):
        crossChr1 = Chromosome(crossoverSinglePoint(population_best[u-1], population_best[u]))
        crossChr2 = Chromosome(crossoverSinglePoint(population_best[u], population_best[u-1]))
        child = np.append(child, crossChr1)
        child = np.append(child, crossChr2)
    population = child
    for mutIter in range(0, len(population)-1):
        rand = random.random()
        if rand < 0.2:
            population[mutIter].mutationGene(0.1)
   # bestChromoIter = population[1]
   # newPopulation = np.append(newPopulation,bestChromoIter)
    print(np.max(fitList))
    X_opti = np.append(X_opti, z)
    Y_opti = np.append(Y_opti, np.max(fitList))

print(newPopulation)


In [ ]:
X_opti

In [ ]:
plt.plot(X_opti, Y_opti)
plt.show()

In [ ]:
len(population_best[0].pathWay(shortestMulti))

In [ ]:
len(np.unique(population_best[0].pathWay(shortestMulti)))

In [ ]:
13 * 27

